In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationSummaryBufferMemory

In [26]:
llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-3.5-turbo",
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [37]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
    )

In [38]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [39]:
examples = [
    {
        "movie": "Avengers: Endgame",
        "answer": "🦸‍♂️🧪🕰️"
     },
     {
        "movie": "The Lion King",
        "answer":"🦁👑🌅"
     },
     {
        "movie": "Top Gun: Maverick",
        "answer": "🛩️👨‍✈️🔥"
     }
]

In [40]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human","{movie}"),
        ("ai","{answer}"),
    ],
)

fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a Ph.D degree in Movie. And you are insane at movie."),
        MessagesPlaceholder(variable_name="history"),
        fewshot_prompt,
        ("human", "{movie}"),
    ]
)

In [54]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(movie):
    result = chain.invoke({"movie":movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )

In [55]:
invoke_chain("Moana")

🌊🚣‍♀️🌺The human mentions "Moana" and the AI responds with emojis of the ocean, a boat, and a flower. The human then brings up "The Dark Knight" and the AI responds with emojis of a bat, a joker, and a city skyline. When the human mentions "Moana" again, the AI seems to be connecting the two movies through emojis by referencing the ocean, boat, and flower. The human mentions "The Dark Knight" and the AI responds with emojis of a bat, a joker, and a city skyline. The AI responds to "Moana" with emojis of the ocean, a boat, and a flower.

In [56]:
invoke_chain("The Dark Knight")

🦇🃏🌆The human mentions "Moana" and the AI responds with emojis of the ocean, a boat, and a flower. The human then brings up "The Dark Knight" and the AI responds with emojis of a bat, a joker, and a city skyline. The AI seems to be connecting the two movies through emojis. The human mentions "The Dark Knight" again, and the AI responds with emojis of a bat, a joker, and a city skyline. The AI responds to "Moana" with emojis of the ocean, a boat, and a flower. When the human mentions "The Dark Knight," the AI responds with emojis of a bat, a joker, and a city skyline.

In [57]:
invoke_chain("Titanic")

🚢❤️🌊The human mentions "Moana" and the AI responds with emojis of the ocean, a boat, and a flower. The AI seems to be connecting the two movies through emojis.

In [58]:
invoke_chain("The Frozen")

❄️👸⛄The human mentions "Moana" and the AI responds with emojis of the ocean, a boat, and a flower. The AI seems to be connecting the two movies through emojis. When the human brings up "The Dark Knight," the AI responds with emojis of a bat, a joker, and a city skyline.